<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-17 13:58:40
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.43 C
-------------------
Today PnL: 95.62 K (0.67%)
Current PnL: -31.41 L (-19.63%)
CY Booked + Current PnL: -15.73 L (-9.83%)
-------------------
Total profit:  1.24 L
Total loss:  -32.65 L
-------------------
Total Booked + Current PnL: 11.25 L (8.46%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (10.96%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 96.77 L (67.65%)
Deployed:  1.33 C
Current:  1.43 C
CAGR/XIRR %: 3.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.43,3.63,17.31,21.57,17576.0,3555.0,101535.0,397.04,-10.57,60.0,H-SC,4.33,167.0,0.20,0.74,48.17,XR,ATH,FINANCE
18,COALINDIA,-0.21,3.68,15.00,19.23,23399.0,5532.0,155992.0,484.83,24.82,49.0,L-LC,11.26,185.0,0.24,1.13,23.68,XY25,ATH,MINING
77,TTKPRESTIG,-0.78,-28.01,39.03,0.09,28318.0,-28223.0,72554.0,770.00,68.41,37.0,M-SC,9.15,250.0,-1.00,0.53,0.00,OX40N,NTT,DURABLES
51,MEDANTA,1.24,-3.20,28.18,24.08,34629.0,-4064.0,122886.0,1486.00,-3.93,55.0,X-SC,11.46,87.0,-0.12,0.89,12.55,XY24,NTT,HEALTHCARE
36,ICICIGI,0.21,2.02,16.97,19.34,34645.0,4047.0,204156.0,2252.93,-14.19,59.0,X-MC,3.03,66.0,0.12,1.48,18.28,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,19.95,-47.57,177.89,45.70,164906.0,-84114.0,92701.0,26.40,-3.72,81.0,M-SC,24.60,204.0,-0.51,0.67,52.73,XY24,NTT,TRAVEL
83,WHIRLPOOL,3.64,-25.40,143.85,81.92,123197.0,-29155.0,85643.0,2270.00,-56.34,65.0,M-SC,29.63,210.0,-0.24,0.62,20.30,XR,NTT,DURABLES
56,RAJOOENG,2.98,-34.85,114.29,39.61,76323.0,-35720.0,66780.0,143.10,-56.03,52.0,H-SC,28.38,136.0,-0.47,0.48,11.88,AR,ATH,MISC
64,SHALBY,2.75,-39.16,105.61,25.10,116398.0,-70928.0,110215.0,327.00,710.21,36.0,M-SC,17.20,236.0,-0.61,0.80,2.75,XY24,NTT,HEALTHCARE
71,TANLA,2.67,-46.27,308.90,119.69,449363.0,-125289.0,145472.0,1943.92,-45.79,44.0,H-SC,18.97,147.0,-0.28,1.05,14.92,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIS,-2.02,-29.65,72.38,21.26,56533.0,-32926.0,78106.0,528.00,1835.23,34.0,H-SC,4.20,168.0,-0.58,0.57,7.68,OX40N,NTT,MISC
45,JCHAC,-1.55,-39.46,65.16,-0.01,50418.0,-50430.0,77375.0,2282.00,15726.09,48.0,M-SC,17.14,231.0,-1.00,0.56,7.27,OX40N,BTT,AC
50,MASFIN,-1.43,3.63,17.31,21.57,17576.0,3555.0,101535.0,397.04,-10.57,60.0,H-SC,4.33,167.0,0.20,0.74,48.17,XR,ATH,FINANCE
80,VALIANTORG,-1.17,-49.44,244.44,74.15,202198.0,-80886.0,82719.0,838.00,-249.10,41.0,H-SC,24.26,145.0,-0.40,0.60,6.03,XR,NTT,CHEMICALS
55,RAJESHEXPO,-1.14,-67.72,209.99,0.07,92988.0,-92895.0,44282.0,518.00,1522.47,44.0,L-SC,8.98,268.0,-1.00,0.32,10.13,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1.31,3.22,101.43,107.91,154575.0,4752.0,152396.0,1641.55,-7.15,64.0,M-SC,9.48,220.0,0.03,1.10,14.65,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.78,-28.01,39.03,0.09,28318.0,-28223.0,72554.0,770.00,68.41,37.0,M-SC,9.15,250.0,-1.00,0.53,0.00,OX40N,NTT,DURABLES
72,TATAELXSI,2.32,-28.91,85.71,32.03,80333.0,-38108.0,93727.0,9161.00,-11.12,37.0,H-SC,6.86,158.0,-0.47,0.68,4.98,OX40N,NTT,IT
66,SIS,-2.02,-29.65,72.38,21.26,56533.0,-32926.0,78106.0,528.00,1835.23,34.0,H-SC,4.20,168.0,-0.58,0.57,7.68,OX40N,NTT,MISC
40,INDIGOPNTS,-0.82,-30.43,43.78,0.02,53163.0,-53126.0,121433.0,1408.00,85.27,31.0,M-SC,8.34,222.0,-1.00,0.88,5.63,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.21,3.68,15.00,19.23,23399.0,5532.0,155992.0,484.83,24.82,49.0,L-LC,11.26,185.0,0.24,1.13,23.68,XY25,ATH,MINING
26,FINCABLES,1.31,3.22,101.43,107.91,154575.0,4752.0,152396.0,1641.55,-7.15,64.0,M-SC,9.48,220.0,0.03,1.10,14.65,OX40N,ATH,CABLES
50,MASFIN,-1.43,3.63,17.31,21.57,17576.0,3555.0,101535.0,397.04,-10.57,60.0,H-SC,4.33,167.0,0.20,0.74,48.17,XR,ATH,FINANCE
1,ABB,-0.93,7.03,36.01,45.57,100831.0,18389.0,280008.0,7934.00,-31.20,68.0,H-MC,3.53,125.0,0.18,2.03,24.34,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.43,3.63,17.31,21.57,17576.0,3555.0,101535.0,397.04,-10.57,60.0,H-SC,4.33,167.0,0.20,0.74,48.17,XR,ATH,FINANCE
1,ABB,-0.93,7.03,36.01,45.57,100831.0,18389.0,280008.0,7934.00,-31.20,68.0,H-MC,3.53,125.0,0.18,2.03,24.34,AR,NTT,ELECTRICAL
26,FINCABLES,1.31,3.22,101.43,107.91,154575.0,4752.0,152396.0,1641.55,-7.15,64.0,M-SC,9.48,220.0,0.03,1.10,14.65,OX40N,ATH,CABLES
39,INDIAMART,1.09,-4.18,115.73,106.72,136776.0,-5151.0,118185.0,4810.62,-56.84,51.0,H-SC,7.72,140.0,-0.04,0.86,18.22,AR,ATH,MISC
85,ZYDUSLIFE,-0.60,-4.57,42.12,35.63,84242.0,-9569.0,200005.0,1286.17,-17.12,51.0,H-MC,5.61,120.0,-0.11,1.45,11.34,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,2.63,-4.59,40.71,34.24,121527.0,-14368.0,298520.0,1972.00,-24.89,28.0,X-LC,4.27,3.0,-0.12,2.16,3.31,X40,NTT,IT
73,TCS,0.86,-24.94,57.93,18.55,164475.0,-94317.0,283920.0,4311.59,-32.15,29.0,X-LC,3.66,2.0,-0.57,2.06,1.40,X40,ATH,IT
33,HCLTECH,1.78,-3.45,28.26,23.83,66011.0,-8339.0,233585.0,1908.19,1.32,30.0,X-LC,4.82,8.0,-0.13,1.69,12.07,X40,ATH,IT
8,AWL,-0.15,-34.01,138.21,57.19,297435.0,-110922.0,215205.0,485.00,-62.20,30.0,X-SC,17.42,81.0,-0.37,1.56,1.01,XY24,NTT,FMCG
13,BERGEPAINT,0.22,-17.45,46.74,21.14,87720.0,-39662.0,187677.0,680.00,-19.19,31.0,X-MC,11.93,68.0,-0.45,1.36,1.07,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,0.34,0.88,42.44,43.70,106210.0,2192.0,250260.0,735.00,-8.39,54.0,X-MC,1.83,70.0,0.02,1.81,16.07,XY24,BTT,FMCG
65,SIEMENS,-0.83,-14.32,46.49,25.51,74126.0,-26650.0,159445.0,4671.50,33.99,56.0,H-LC,1.93,51.0,-0.36,1.15,17.50,AR,ATH,ELECTRICAL
78,UNITDSPR,1.20,4.15,15.95,20.77,38896.0,9718.0,243862.0,1644.00,-7.70,63.0,X-MC,1.96,64.0,0.25,1.77,11.86,X40N,NTT,BREWERIES
34,HINDUNILVR,-0.52,-10.26,26.66,13.67,61505.0,-26363.0,230700.0,2922.00,-33.31,40.0,X-LC,2.87,9.0,-0.43,1.67,8.91,XY25,NTT,FMCG
36,ICICIGI,0.21,2.02,16.97,19.34,34645.0,4047.0,204156.0,2252.93,-14.19,59.0,X-MC,3.03,66.0,0.12,1.48,18.28,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,0.26,-13.08,51.87,32.01,96271.0,-27920.0,185600.0,17616.87,-41.22,35.0,X-MC,9.32,57.0,-0.29,1.34,0.32,X40,ATH,FMCG
3,ACC,0.56,-31.02,137.97,64.15,226464.0,-73811.0,164140.0,3906.00,-48.05,40.0,X-SC,8.36,85.0,-0.33,1.19,0.56,XY24,BTT,CEMENT
2,ABBOTINDIA,0.80,-11.10,32.81,18.07,52168.0,-19855.0,159000.0,35195.00,-24.45,38.0,X-MC,10.48,58.0,-0.38,1.15,0.80,X40,ATH,PHARMA
9,BAJAJHFL,-0.09,-23.98,104.51,55.47,193296.0,-58331.0,184955.0,181.50,-34.73,37.0,X-MC,17.39,60.0,-0.30,1.34,0.83,X40N,ATH,FINANCE
8,AWL,-0.15,-34.01,138.21,57.19,297435.0,-110922.0,215205.0,485.00,-62.20,30.0,X-SC,17.42,81.0,-0.37,1.56,1.01,XY24,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.18,-30.12,104.41,42.84,47430.0,-19579.0,45427.0,424.00,-54.27,51.0,X-SC,19.44,80.0,-0.41,0.33,9.27,XY24,NTT,MISC
58,RELAXO,2.44,-52.93,223.57,52.31,152763.0,-76831.0,68329.0,1176.00,-49.86,41.0,X-SC,14.17,92.0,-0.50,0.49,2.44,X40N,NTT,FOOTWEAR
12,BATAINDIA,1.23,-45.88,149.81,35.20,104329.0,-59029.0,69641.0,2096.00,-8.74,37.0,X-SC,21.42,91.0,-0.57,0.50,1.23,X40,NTT,FOOTWEAR
51,MEDANTA,1.24,-3.20,28.18,24.08,34629.0,-4064.0,122886.0,1486.00,-3.93,55.0,X-SC,11.46,87.0,-0.12,0.89,12.55,XY24,NTT,HEALTHCARE
35,HONAUT,0.53,-22.98,86.09,43.32,107991.0,-37432.0,125440.0,58357.33,-32.37,39.0,X-SC,10.92,90.0,-0.35,0.91,1.85,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,2.30,-20.28,38.97,10.78,74656.0,-48746.0,191572.0,452.00,-53.09,49.0,X-LC,18.82,1.0,-0.65,1.39,5.52,X40,NTT,FMCG
73,TCS,0.86,-24.94,57.93,18.55,164475.0,-94317.0,283920.0,4311.59,-32.15,29.0,X-LC,3.66,2.0,-0.57,2.06,1.40,X40,ATH,IT
42,INFY,2.63,-4.59,40.71,34.24,121527.0,-14368.0,298520.0,1972.00,-24.89,28.0,X-LC,4.27,3.0,-0.12,2.16,3.31,X40,NTT,IT
75,TMPV,0.99,-21.91,57.48,22.98,101396.0,-49495.0,176403.0,600.00,-82.53,63.0,X-LC,3.11,4.0,-0.49,1.28,12.77,XY24,NTT,AUTO
33,HCLTECH,1.78,-3.45,28.26,23.83,66011.0,-8339.0,233585.0,1908.19,1.32,30.0,X-LC,4.82,8.0,-0.13,1.69,12.07,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.97,-30.53,91.30,32.89,48968.0,-23572.0,53634.0,1800.00,-352.80,64.0,L-MC,10.88,258.0,-0.48,0.39,47.70,XR,NTT,BANKS
25,EASEMYTRIP,19.95,-47.57,177.89,45.70,164906.0,-84114.0,92701.0,26.40,-3.72,81.0,M-SC,24.60,204.0,-0.51,0.67,52.73,XY24,NTT,TRAVEL
67,SONACOMS,0.83,-8.41,51.86,39.09,48050.0,-8505.0,92654.0,804.02,-27.36,63.0,M-MC,6.01,193.0,-0.18,0.67,31.07,AR,ATH,AUTO
50,MASFIN,-1.43,3.63,17.31,21.57,17576.0,3555.0,101535.0,397.04,-10.57,60.0,H-SC,4.33,167.0,0.20,0.74,48.17,XR,ATH,FINANCE
79,VAIBHAVGBL,2.16,-22.41,107.19,60.77,152020.0,-40952.0,141823.0,521.00,63.64,58.0,H-SC,0.76,160.0,-0.27,1.03,28.58,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,19.95,-47.57,177.89,45.70,164906.0,-84114.0,92701.0,26.40,-3.72,81.0,M-SC,24.60,204.0,-0.51,0.67,52.73,XY24,NTT,TRAVEL
1,ABB,-0.93,7.03,36.01,45.57,100831.0,18389.0,280008.0,7934.00,-31.20,68.0,H-MC,3.53,125.0,0.18,2.03,24.34,AR,NTT,ELECTRICAL
79,VAIBHAVGBL,2.16,-22.41,107.19,60.77,152020.0,-40952.0,141823.0,521.00,63.64,58.0,H-SC,0.76,160.0,-0.27,1.03,28.58,XR,NTT,JEWELLERY
67,SONACOMS,0.83,-8.41,51.86,39.09,48050.0,-8505.0,92654.0,804.02,-27.36,63.0,M-MC,6.01,193.0,-0.18,0.67,31.07,AR,ATH,AUTO
11,BANDHANBNK,0.47,-18.84,137.08,92.42,309917.0,-52474.0,226085.0,400.00,93.97,69.0,H-SC,7.48,174.0,-0.17,1.64,30.66,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.24
1,25,44.64
2,50,76.70


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.32
MC    30.33
LC    23.33
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.83
X40      23.25
X40N     15.17
AR        9.02
XR        8.78
XY25      8.75
OX40N     7.30
SR        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.91
H-SC    22.72
X-LC    19.96
X-SC    11.81
M-SC    10.99
H-MC     4.94
M-MC     2.09
H-LC     1.15
L-LC     1.13
M-LC     1.09
L-SC     0.80
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.00,-14.32,52.73
FINANCE,13.37,-13.20,57.75
IT,11.65,-32.81,99.08
MISC,7.10,-32.64,86.90
ELECTRICAL,6.27,-8.44,48.00
PAINTS,5.11,-30.90,50.35
INSURANCE,4.86,-1.09,35.68
PHARMA,4.12,-3.44,36.00
AUTO,3.18,-19.26,60.50


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3265978.0,21
AR,1324516.0,10
X40,1287468.0,15
XR,1279741.0,12
X40N,1026679.0,10
OX40N,784681.0,10
XY25,410436.0,6
SR,297372.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3654900.0,24
M-SC,1399026.0,14
X-MC,1350855.0,15
X-SC,1172514.0,10
X-LC,1137101.0,12
H-MC,396629.0,3
L-SC,180979.0,2
M-LC,141562.0,1
M-MC,96812.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1242692.0      6
           AR         943473.0      5
           XR         824645.0      7
M-SC       XY24       803682.0      6
X-SC       XY24       605958.0      4
X-LC       X40        572874.0      6
X-MC       X40        538924.0      7
           X40N       409544.0      4
X-SC       X40N       390886.0      4
H-SC       OX40N      346718.0      4
M-SC       OX40N      344975.0      5
H-SC       SR         297372.0      2
X-LC       X40N       226249.0      2
H-MC       XY24       211556.0      1
X-MC       XY25       208457.0      2
X-LC       XY24       208160.0      2
X-MC       XY24       193930.0      2
H-MC       AR         185073.0      2
M-SC       XR         176575.0      2
X-SC       X40        175670.0      2
M-LC       XR         141562.0      1
X-LC       XY25       129818.0      2
L-SC       OX40N       92988.0      1
           XR          87991.0      1
H-LC       AR          74126.0      1
M-SC       AR          73794.0      1
L-MC       XR          48968.0      1
M-MC       XY25        48762.0      1
           AR          48050.0      1
L-LC       XY25        23399.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 23.0 seconds
